<a href="https://colab.research.google.com/github/Periato/checkpoint2-energiarenovavel/blob/main/Checkpoint_2_Andr%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Diego - 565509

Israel - 563435

Pedro Miranda - 562117

Thiago Sakai - 563448

Thiago Sobral - 562695

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report

# Exercício 1 – Classificação (Solar)

In [ ]:

# 1. Carregar dataset
df = pd.read_csv("/content/SolarPrediction.csv")

# 2. Criar variável-alvo binária (0 = baixa, 1 = alta radiação)
limiar = df["Radiation"].median()
df["Target"] = (df["Radiation"] > limiar).astype(int)

# 3. Selecionar atributos preditores (sem colunas de tempo)
X = df[["Temperature", "Pressure", "Humidity",
        "WindDirection(Degrees)", "Speed"]]
y = df["Target"]

# 4. Separar treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 5. Normalizar atributos contínuos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ================================
# 6. Treinar Modelos
# ================================

# Árvore de Decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

# Random Forest
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# SVM (com dados normalizados)
svm = SVC(random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)

# ================================
# 7. Avaliar Modelos
# ================================

def avaliar_modelo(nome, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nModelo: {nome}")
    print(f"Acurácia: {acc:.4f}")
    print("Matriz de Confusão:")
    print(cm)

avaliar_modelo("Árvore de Decisão", y_test, y_pred_tree)
avaliar_modelo("Random Forest", y_test, y_pred_rf)
avaliar_modelo("SVM", y_test, y_pred_svm)



Modelo: Árvore de Decisão
Acurácia: 0.8353
Matriz de Confusão:
[[4057  849]
 [ 766 4134]]

Modelo: Random Forest
Acurácia: 0.8822
Matriz de Confusão:
[[4566  340]
 [ 815 4085]]

Modelo: SVM
Acurácia: 0.8363
Matriz de Confusão:
[[4642  264]
 [1341 3559]]


# Exercício 2 – Regressão (Eólica)

In [ ]:
# 1. Carregar os dados
df = pd.read_csv("/content/T1.csv")

# 2. Selecionar variáveis
X = df[["Wind Speed (m/s)", "Theoretical_Power_Curve (KWh)", "Wind Direction (°)"]]
y = df["LV ActivePower (kW)"]

# 3. Dividir em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Normalizar atributos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ================================
# 5. Treinar Modelos
# ================================

# Regressão Linear
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)

# Árvore de Regressão
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)
y_pred_tree = tree_reg.predict(X_test)

# Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42, n_estimators=100)
rf_reg.fit(X_train, y_train)
y_pred_rf = rf_reg.predict(X_test)

# ================================
# 6. Avaliar Modelos
# ================================

def avaliar_modelo(nome, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\nModelo: {nome}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")

avaliar_modelo("Regressão Linear", y_test, y_pred_lin)
avaliar_modelo("Árvore de Regressão", y_test, y_pred_tree)
avaliar_modelo("Random Forest Regressor", y_test, y_pred_rf)



Modelo: Regressão Linear
RMSE: 411.71
R²: 0.9007

Modelo: Árvore de Regressão
RMSE: 538.79
R²: 0.8299

Modelo: Random Forest Regressor
RMSE: 412.74
R²: 0.9002


# Parte 1 - Regressão (Appliances Energy Prediction)

In [ ]:
# Ajuste o nome do arquivo conforme o dataset dentro do zip
df = pd.read_csv("/content/energydata_complete.csv")
print(df.head())
print(df.info())

# Drop the 'date' column as it's not a numerical feature
X = df.drop(columns=["Appliances", "date"])  # variáveis preditoras
y = df["Appliances"]                 # variável alvo (consumo Wh)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
modelos = {
    "Regressão Linear": LinearRegression(),
    "Árvore de Regressão": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

resultados = {}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    resultados[nome] = {
        "R²": r2_score(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "MAE": mean_absolute_error(y_test, y_pred)
    }

pd.DataFrame(resultados).T

                  date  Appliances  lights     T1       RH_1    T2       RH_2  \
0  2016-01-11 17:00:00          60      30  19.89  47.596667  19.2  44.790000   
1  2016-01-11 17:10:00          60      30  19.89  46.693333  19.2  44.722500   
2  2016-01-11 17:20:00          50      30  19.89  46.300000  19.2  44.626667   
3  2016-01-11 17:30:00          50      40  19.89  46.066667  19.2  44.590000   
4  2016-01-11 17:40:00          60      40  19.89  46.333333  19.2  44.530000   

      T3       RH_3         T4  ...         T9   RH_9     T_out  Press_mm_hg  \
0  19.79  44.730000  19.000000  ...  17.033333  45.53  6.600000        733.5   
1  19.79  44.790000  19.000000  ...  17.066667  45.56  6.483333        733.6   
2  19.79  44.933333  18.926667  ...  17.000000  45.50  6.366667        733.7   
3  19.79  45.000000  18.890000  ...  17.000000  45.40  6.250000        733.8   
4  19.79  45.000000  18.890000  ...  17.000000  45.40  6.133333        733.9   

   RH_out  Windspeed  Visibility

,R²,RMSE,MAE
Regressão Linear,0.169304,91.174826,52.547693
Árvore de Regressão,0.179809,90.596509,38.872561
Random Forest,0.532236,68.417501,32.900228


# Parte 2 - Classificação (Smart Grid Stability)

In [ ]:
df2 = pd.read_csv("/content/smart_grid_stability_augmented.csv")
print(df2.head())
print(df2['stabf'].value_counts())  # alvo é "stabf" (stable/unstable)

X = df2.drop(columns=["stabf"])
y = df2["stabf"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

modelos_class = {
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Regressão Logística": LogisticRegression(max_iter=1000)
}

resultados_class = {}

for nome, modelo in modelos_class.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    resultados_class[nome] = {
        "Acurácia": accuracy_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred, average="weighted"),
        "Matriz de Confusão": confusion_matrix(y_test, y_pred)
    }

for nome, res in resultados_class.items():
    print(f"\nModelo: {nome}")
    print("Acurácia:", res["Acurácia"])
    print("F1-score:", res["F1-score"])
    print("Matriz de Confusão:\n", res["Matriz de Confusão"])



       tau1      tau2      tau3      tau4        p1        p2        p3  \
0  2.959060  3.079885  8.381025  9.780754  3.763085 -0.782604 -1.257395   
1  9.304097  4.902524  3.047541  1.369357  5.067812 -1.940058 -1.872742   
2  8.971707  8.848428  3.046479  1.214518  3.405158 -1.207456 -1.277210   
3  0.716415  7.669600  4.486641  2.340563  3.963791 -1.027473 -1.938944   
4  3.134112  7.608772  4.943759  9.857573  3.525811 -1.125531 -1.845975   

         p4        g1        g2        g3        g4      stab     stabf  
0 -1.723086  0.650456  0.859578  0.887445  0.958034  0.055347  unstable  
1 -1.255012  0.413441  0.862414  0.562139  0.781760 -0.005957    stable  
2 -0.920492  0.163041  0.766689  0.839444  0.109853  0.003471  unstable  
3 -0.997374  0.446209  0.976744  0.929381  0.362718  0.028871  unstable  
4 -0.554305  0.797110  0.455450  0.656947  0.820923  0.049860  unstable  
stabf
unstable    38280
stable      21720
Name: count, dtype: int64

Modelo: Árvore de Decisão
Acurácia: 